In [1]:
from nnsight.models.UnifiedTransformer import UnifiedTransformer

model = UnifiedTransformer(
    'gpt2-small',
    processing=False,
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device="cuda",
)
tokenizer = model.tokenizer

model.local_model.set_use_hook_mlp_in(True)
model.local_model.set_use_split_qkv_input(True)
model.local_model.set_use_attn_result(True)
model.update_meta()

print(model.blocks[0].attn.hook_k.input)


/share/u/caden/miniconda3/envs/interp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  meta
Moving model to device:  meta


AttributeError: 'HookPoint' object has no attribute 'input'

In [16]:
from nnsight import LanguageModel
import torch as t

model = LanguageModel("gpt2", device_map="auto", dispatch=True)
tokenizer = model.tokenizer

print(model.transformer.h[0].attn.c_attn.input.shape)

AttributeError: 'Conv1D' object has no attribute 'input'

In [15]:
model.transformer.h[0].ln_1

LayerNorm((768,), eps=1e-05, elementwise_affine=True)

In [3]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [2]:
with model.forward(remote=REMOTE) as runner:

    # Run forward pass and store each attention head output 
    attn_dict = {} # [d_head]
    with runner.invoke("test") as invoker:
        for layer in 12:
            for pos in range(seq_len-6, seq_len):
                
                assert seq_len == model.model.layers[layer].output[0].shape[1], f"{seq_len=} != {model.model.layers[layer].output[0].shape[1]=}, {model.model.layers[layer].output[0].shape=}"
                z = model.model.layers[layer].self_attn.input[1]["hidden_states"][:, pos]

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  meta
Moving model to device:  meta
